# task image classifier for itsaaz  company 

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import pathlib 

In [2]:
import tensorflow as tf 

In [ ]:
!unzip '/content/drive/MyDrive/data.zip'

In [25]:
directory='/content/data'
train_dir=pathlib.Path(directory)
ClassName=np.array([item.name for item in train_dir.glob('*')])
Full_dataset=tf.data.Dataset.list_files(str(train_dir/'*/*')) # All image training

In [26]:
ClassName

array(['croped', 'sign', 'False', 'stamp'], dtype='<U6')

In [4]:
validation_split=0.1
Dataset_size=len(list(Full_dataset))
train_size=int((1-validation_split)*Dataset_size)
train_dataset=Full_dataset.take(train_size)
validation_dataset=Full_dataset.skip(train_size)

##  * preprocess data

# create helper function 

In [5]:
def augment_data(image):
  image=tf.image.resize_with_crop_or_pad(image,180,180)
  image=tf.image.random_crop(image,size=[150,150,3])
  image=tf.image.random_brightness(image,max_delta=0.5)
  return image

def get_label(file_path):
  parts=tf.strings.split(file_path,os.path.sep)
  return parts[-2]==ClassName

def load_image(image_path):
  image=tf.io.read_file(image_path)
  image=tf.image.decode_jpeg(image,3)
  image=tf.cast(image,tf.float32)
  return image 

def normalize(image):
  image=(image/127.5)-1
  return image 

def resize(image,height,width):
  image=tf.image.resize(image,(height,width),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  return image 

# creat base function 

In [6]:
def load_image_with_label(image_path):
  label=get_label(image_path)
  img=load_image(image_path)
  return img,label

def load_image_train(image_file):
  image,label=load_image_with_label(image_file)
  image=augment_data(image)
  image=normalize(image)
  return image,label

def load_image_test(image_file):
  image,label=load_image_with_label(image_file)
  image=resize(image,150,150)
  image=normalize(image)
  return image,label


# create pipeline dataset 

In [7]:
train_dataset=train_dataset.map(load_image_train)
train_dataset=train_dataset.shuffle(1000)
train_dataset=train_dataset.batch(32)

validation_dataset=validation_dataset.map(load_image_test)
validation_dataset=validation_dataset.batch(32)

## * creat model

ما در این پروزه به دلیل این که تعداد داده های محدودی برای کلاس بندی داریم مجبور هستیم از شبکه های از پیش اموزش دیده استفاده کنیم 

In [8]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Flatten,Input
from tensorflow.keras.models import Model

# creat model with API function 

In [9]:
base_Conv=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(150,150,3)))

58900480/58889256 [==============================] - 1s 0us/step


In [10]:
base_Conv.trainable=False

In [20]:
inputs=base_Conv.output
layer=Flatten()(inputs)
layer=Dense(512,activation='relu')(layer)
layer=BatchNormalization()(layer)
layer=Dropout(rate=0.25)(layer)
layer=Dense(128,activation='relu')(layer)
layer=Dropout(rate=0.25)(layer)
output=Dense(4,activation='softmax')(layer)

model=Model(base_Conv.input,output)

In [21]:
model.compile(loss=CategoricalCrossentropy(),optimizer='adam',metrics=['acc'])

creat checkpoint

In [22]:
file_path='/content/my_model.h5'
checkpoint=tf.keras.callbacks.ModelCheckpoint(
                    file_path,
                    monitor='val_loss',
                    save_best_only=True)
callback_list=[checkpoint]

In [23]:
hi=model.fit(train_dataset,epochs=15,validation_data=validation_dataset,
             callbacks=callback_list)

Epoch 1/15
9/9 [==============================] - 11s 249ms/step - loss: 1.3697 - acc: 0.5017 - val_loss: 1.2082 - val_acc: 0.5312
Epoch 2/15
9/9 [==============================] - 9s 144ms/step - loss: 0.7247 - acc: 0.7387 - val_loss: 1.5473 - val_acc: 0.4375
Epoch 3/15
9/9 [==============================] - 9s 140ms/step - loss: 0.4663 - acc: 0.8258 - val_loss: 1.6930 - val_acc: 0.5625
Epoch 4/15
9/9 [==============================] - 11s 207ms/step - loss: 0.4460 - acc: 0.8397 - val_loss: 1.2714 - val_acc: 0.6562
Epoch 5/15
9/9 [==============================] - 9s 149ms/step - loss: 0.3292 - acc: 0.8955 - val_loss: 1.9570 - val_acc: 0.4062
Epoch 6/15
9/9 [==============================] - 9s 139ms/step - loss: 0.2919 - acc: 0.8885 - val_loss: 1.8717 - val_acc: 0.5938
Epoch 7/15
9/9 [==============================] - 9s 154ms/step - loss: 0.2367 - acc: 0.9233 - val_loss: 1.6285 - val_acc: 0.5938
Epoch 8/15
9/9 [==============================] - 9s 142ms/step - loss: 0.2200 - acc: 0.